In [1]:
import pandas as pd
from evaluate import load

In [2]:
just_df = pd.read_pickle('justifications.pkl')

In [3]:
just_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,claim_kg_size,main_text_kg_size,clean_claim_triples,clean_main_text_triples,claim_evidence_1,entailment_scores_1,verdict,verdict_actual,justification_base,justification_mtl
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,...,3,14,"[(A baby died at an unnamed medical facility, ...","[(Confederate flag, instance of, racist), (Fie...","{'(A baby died at an unnamed medical facility,...","[[0.66573095, -0.56622183, -0.4859991], [0.430...",-1,0,A baby died at an unnamed medical facility and...,A baby died at an unnamed medical facility bec...
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...",...,3,6,"[(Shawnee County, located in the administrativ...","[(KSNT, located in the administrative territor...","{'(Shawnee County, located in the administrati...","[[0.6284045, -0.64509016, -0.4347026], [0.6175...",-1,1,"Because Topeka is located in Shawnee County, i...",Topeka is not located in Shawnee County. Bat f...
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...",...,3,36,"[(banned pork from school canteens, country, G...","[(Express, country, British), (Express, instan...","{'(banned pork from school canteens, country, ...","[[0.64209604, -0.69231135, -0.32926825], [0.68...",-1,-1,Germany banned pork from school canteens becau...,Germany banned pork from school canteen becaus...
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...",...,3,21,"[(Coronavirus, country, Canada), (safe drugs, ...","[(heroin, instance of, controlled substance), ...","{'(Coronavirus, country, Canada)': ['(coronavi...","[[-0.1503326, -0.75800896, 0.63468295], [0.660...",-1,1,The Coronavirus is in Canada and is unrelated ...,Coronavirus is found in Canada and is unrelate...
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...",...,3,3,"[(Wayne National Forest, instance of, wildlife...","[(oak forests, located in the administrative t...","{'(Wayne National Forest, instance of, wildlif...","[[0.63135314, -0.7029956, -0.32739952], [0.662...",-1,1,The Wayne National Forest is not an example of...,The Wayne National Forest is not an instance o...


In [4]:
bertscore = load("bertscore")
rouge = load('rouge')
sacrebleu = load('sacrebleu')

def get_metrics(predictions, references, references_sb):
    results = {}    
    results["bertscore"] = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
    results["rouge"] = rouge.compute(predictions=predictions, references=references)
    results["sacrebleu"] = sacrebleu.compute(predictions=predictions, references=references_sb)

    return results 

In [5]:
justifications_base = just_df['justification_base'].tolist()
justifications_mtl = just_df['justification_mtl'].tolist()
references = just_df['explanation'].tolist()
references_sb = [[reference] for reference in references]

In [6]:
results_base = get_metrics(justifications_base, references, references_sb)
print(results_base)
results_mtl = get_metrics(justifications_mtl, references, references_sb)
print(results_mtl)

{'bertscore': {'precision': [0.734747052192688, 0.6771665811538696, 0.7503678798675537, 0.7775523066520691, 0.7865230441093445, 0.7506698369979858, 0.7598093748092651, 0.807999849319458, 0.7888888120651245, 0.7148440480232239, 0.8036172986030579, 0.7920911312103271, 0.8166220188140869, 0.7223641872406006, 0.6969635486602783, 0.6119730472564697, 0.7611147165298462, 0.7806648015975952, 0.7166830897331238, 0.8573193550109863, 0.765641450881958, 0.7450829744338989, 0.8072949051856995, 0.8432123064994812, 0.6949896812438965, 0.7487615346908569, 0.7828165292739868, 0.7133558988571167, 0.822844922542572, 0.793100118637085, 0.7386398315429688, 0.773349940776825, 0.81758052110672, 0.7517929673194885, 0.7802050709724426, 0.7266086935997009, 0.7651549577713013, 0.6676973104476929, 0.7906802892684937, 0.7604882717132568, 0.7629306316375732, 0.7853788137435913, 0.7171961069107056, 0.7621201872825623, 0.7248428463935852, 0.8334165811538696, 0.8085103631019592, 0.7182520627975464, 0.8006088137626648,

In [7]:
base_precision = sum(results_base['bertscore']['precision']) / len(results_base['bertscore']['precision'])
base_recall = sum(results_base['bertscore']['recall']) / len(results_base['bertscore']['recall'])
base_f1score = sum(results_base['bertscore']['f1']) / len(results_base['bertscore']['f1'])

print(base_precision, base_recall, base_precision)

0.7707513258420234 0.7164985560865231 0.7707513258420234


In [8]:
mtl_precision = sum(results_mtl['bertscore']['precision']) / len(results_mtl['bertscore']['precision'])
mtl_recall = sum(results_mtl['bertscore']['recall']) / len(results_mtl['bertscore']['recall'])
mtl_f1score = sum(results_mtl['bertscore']['f1']) / len(results_mtl['bertscore']['f1'])

print(mtl_precision, mtl_recall, mtl_f1score)

0.7698669622471438 0.7149430643432876 0.7400893380601099


In [9]:
print(results_base['rouge'])
print(results_mtl['rouge'])

{'rouge1': 0.18262918832160074, 'rouge2': 0.04520613907749116, 'rougeL': 0.13678372095871535, 'rougeLsum': 0.13685863919855681}
{'rouge1': 0.18515373873238616, 'rouge2': 0.04421759632924088, 'rougeL': 0.1384225123327804, 'rougeLsum': 0.13845902315445252}


In [12]:
print(results_base['sacrebleu'])
print(results_mtl['sacrebleu'])

{'score': 0.6246214375368004, 'counts': [22386, 4158, 1211, 440], 'totals': [76251, 73799, 71347, 68895], 'precisions': [29.358303497659048, 5.634222685944254, 1.6973383604075856, 0.6386530227157269], 'bp': 0.17069362239802446, 'sys_len': 76251, 'ref_len': 211054}
{'score': 0.6131975872914872, 'counts': [22547, 4054, 1153, 420], 'totals': [76698, 74246, 71794, 69342], 'precisions': [29.39711596130277, 5.460226813565714, 1.6059837869459843, 0.6056935190793459], 'bp': 0.17346947509517147, 'sys_len': 76698, 'ref_len': 211054}


In [21]:
predictions = ["My name is Prahlad, I study at Columbia"]
references = ["My university is Columbia, I am Prahlad"]
results = sacrebleu.compute(predictions=predictions, references=references, tokenize='none', lowercase=True, use_effective_order=True)
print(results)

{'score': 7.267884212102741, 'counts': [3, 0, 0, 0], 'totals': [8, 7, 6, 5], 'precisions': [37.5, 7.142857142857143, 4.166666666666667, 2.5], 'bp': 1.0, 'sys_len': 8, 'ref_len': 7}


In [22]:
rouge.compute(predictions=predictions, references=references)

{'rouge1': 0.6666666666666666,
 'rouge2': 0.0,
 'rougeL': 0.39999999999999997,
 'rougeLsum': 0.39999999999999997}

In [30]:
import numpy as np
np.argmax(results_base['bertscore']['f1'])

1140

In [29]:
print(just_df.iloc[1140]['claim'])
print(just_df.iloc[1140]['explanation'])
print(just_df.iloc[1140]['label'])
print(just_df.iloc[1140]['justification_base'])

Lois Kolkhorst Says $500 billion in Medicare cuts are coming.
Lois Kolkhorst says $500 billion in Medicare cuts are coming
false
Because cuts in Medicare are coming in 2010. Lois Kolkhorst wrote that $500 billion in Medicare cuts are coming in 2010.


In [35]:
def get_stats(just_df, columns):

    for column in columns:
        just_df[f'{column}_tokens'] = just_df[column].apply(lambda n: len(str(n).split()))

get_stats(just_df, columns=['claim', 'main_text', 'explanation'])

In [67]:
small_just_df = just_df[(just_df['main_text_tokens'] < 100) & (just_df['main_text_tokens'] > 10) ]

In [68]:
small_just_df.shape

(57, 25)

In [69]:
small_just_df.columns

Index(['claim_id', 'claim', 'date_published', 'explanation', 'fact_checkers',
       'main_text', 'sources', 'label', 'subjects', 'claim_kg', 'main_text_kg',
       'Unnamed: 0', 'claim_kg_size', 'main_text_kg_size',
       'clean_claim_triples', 'clean_main_text_triples', 'claim_evidence_1',
       'entailment_scores_1', 'verdict', 'verdict_actual',
       'justification_base', 'justification_mtl', 'claim_tokens',
       'main_text_tokens', 'explanation_tokens'],
      dtype='object')

In [70]:
small_just_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,clean_main_text_triples,claim_evidence_1,entailment_scores_1,verdict,verdict_actual,justification_base,justification_mtl,claim_tokens,main_text_tokens,explanation_tokens
36,5583,Alabama: Case of meningococcal disease in Jack...,NaN,The Alabama Department of Public Health’s Immu...,,"In a news release Tuesday, the department said...",,true,"Immunizations, Meningitis, Health, Alabama, Pu...","[{'head': 'Alabama', 'type': 'contains adminis...",...,"[(rash, subclass of, symptoms), (rash, instanc...","{'(Jackson County, located in the administrati...","[[-0.47511232, -0.49882415, 0.72487426], [-0.1...",1,1,"Scottsboro is located in Jackson County, Alaba...","Scottsboro is located in Jackson County, Alaba...",8,90,34
121,40135,Little Lauren Renee Pingel was in a car accid...,"March 17, 2015",Pray for 3-year-old Lauren Renee Pingel in Ama...,Rich Buhler & Staff,This story is true… and the latest news is tha...,https://twitter.com/intent/tweet?text=Pray+for...,true,Prayers,"[{'head': 'Little Lauren Renee Pingel', 'type'...",...,"[(Lauren, date of birth, December 15, 2000), (...","{'(Lauren Renee Pingel, place of birth, Amaril...","[[-0.30718613, -0.62570554, 0.717028], [0.6203...",1,1,"Lauren Renee Pingel was born in Amarillo, Texa...","Lauren Renee Pingel was born in Amarillo, Texa...",32,69,9
189,7264,Zambia to vaccinate 1 million against cholera ...,NaN,Zambia is set to vaccinate one million people ...,,The World Health Organization says doses of th...,,true,"Health, Cholera, Zambia, Africa, Southern Afri...","[{'head': 'outbreak', 'type': 'has cause', 'ta...",...,"[(Cholera, subclass of, acute diarrheal), (Lus...","{'(outbreak, has cause, cholera)': ['(Cholera,...","[[-0.6810702, -0.19560099, 0.70560867], [0.629...",-1,1,The outbreak of acute diarrheal is unrelated t...,An outbreak of cholera is unrelated to acute d...,9,93,18
190,5834,State to labs: report all results of tests for...,NaN,Louisiana’s Department of Health says laborato...,,Louisiana has very high rates of all three dis...,http://ldh.la.gov/index.cfm/newsroom/detail/5162,true,"Health, Hepatitis, Syphilis, Louisiana, Public...","[{'head': 'tests', 'type': 'facet of', 'tail':...",...,"[(Centers for Disease Control and Prevention, ...","{'(tests, facet of, diseases)': ['(Centers for...","[[-0.8129392, 0.15043366, 0.56258297], [0.0264...",1,1,The Centers for Disease Control and Prevention...,The CDC is unrelated to Centers for Disease Co...,11,96,27
237,4952,Health care worker in Grant County has mumps.,NaN,The Grant County Health District has issued an...,,The health district says a vaccinated health c...,http://www.kxly.com/,true,"Health, Moses Lake, Mumps",[{'head': 'Health care worker in Grant County'...,...,"[(KXLY-TV, headquarters location, Spokane), (K...","{'(Grant County, contains administrative terri...","[[-0.6104199, -0.3184183, 0.7252568], [0.60828...",1,1,"Samaritan Healthcare is located in Moses Lake,...",Samaritan Healthcare is located in Moses Lake ...,8,84,20


In [80]:
which_just = small_just_df.iloc[34]
print(which_just['claim'])
print(which_just['main_text'])
print(which_just['explanation'])
print(which_just['label'])
print(which_just['claim_kg'])
print(which_just['main_text_kg'])
print(which_just['clean_claim_triples'])
print(which_just['clean_main_text_triples'])
print(which_just['claim_evidence_1'])
print(which_just['entailment_scores_1'])
print(which_just['verdict'])
print(which_just['justification_base'])

Ohio officials say tick-borne Lyme Disease still a threat.
The state agency reports that 241 cases of Lyme disease have been reported in 60 of Ohio’s 88 counties this year. The number of cases has risen steadily the last five years, with 160 reported last year after only 93 in 2013. Lyme disease is passed to humans by deer ticks and can cause flu-like illnesses, muscle pain and headaches. A rash shaped like a bull’s-eye is often seen around tick bites. The Ohio Department of Natural Resources recommends people walk in the middle of trails and tuck in clothing to prevent bites.
The Ohio Department of Health says hunters and those trekking in wooded areas should use bug spray and take other precautions even in cold weather to prevent being infected with tick-borne Lyme disease.
true
[{'head': 'Lyme Disease', 'type': 'has cause', 'tail': 'tick-borne', 'meta': {'spans': [[0, 128]]}}, {'head': 'Lyme Disease', 'type': 'subclass of', 'tail': 'tick-borne', 'meta': {'spans': [[0, 128]]}}, {'hea

In [54]:
print(small_just_df.iloc[100]['claim'])
print(small_just_df.iloc[100]['explanation'])
print(small_just_df.iloc[100]['label'])
print(small_just_df.iloc[100]['claim_kg'])
print(small_just_df.iloc[100]['clean_claim_triples'])
print(small_just_df.iloc[100]['clean_main_text_triples'])
print(small_just_df.iloc[100]['justification_base'])

Zambia to vaccinate 1 million against cholera amid outbreak.
Zambia is set to vaccinate one million people in its capital against cholera as a deadly outbreak grows.
true
[{'head': 'outbreak', 'type': 'has cause', 'tail': 'cholera', 'meta': {'spans': [[0, 128]]}}, {'head': 'outbreak', 'type': 'country', 'tail': 'Zambia', 'meta': {'spans': [[0, 128]]}}, {'head': 'vaccine', 'type': 'country', 'tail': 'Zambia', 'meta': {'spans': [[0, 128]]}}]
['(outbreak, has cause, cholera)', '(outbreak, country, Zambia)', '(vaccine, country, Zambia)']
['(Cholera, subclass of, acute diarrheal)', '(Lusaka, country, Zambia)', '(Zambia, capital, Lusaka)']
The outbreak of acute diarrheal is unrelated to cholera. Because Lusaka is in Zambia, the outbreak is not from Zambia. Because Lusaka is in Zambia, the vaccine is not from Zambia.
